In [1]:
# Shhhhhhh......
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [2]:
# A simple training model using only the closing price by training it on the last 30 prices and deciding if it should buy or sell tomorrow

In [3]:
# Config Variables
# We'll do one at a time for now instead of just looping through mongoDB for all of them.
company = "MSFT"

# How many days in the future should we look? Aka how many more columns should be added? 
days = 300

# How often should a robot wait to trade?
trade_day_cooldown = 1

In [4]:
import ezmongo
my_df = ezmongo.get_data("GOOG","cleaned")

In [5]:
# With only the date and the closing date, we will generate a new column that will be 1 or 0 which will be whether or not it should sell.

def generate_training_col(df):
    row_count = df.shape[0]  
    
    # This lets us loop through every row of the DF
    for i in df.index:
        if (i < (row_count - trade_day_cooldown)):
            val_now = df.get_value(i,'Close')
            val_next = df.get_value(i+trade_day_cooldown,'Close')
            if val_next > val_now:
                df.set_value( i,'BuyOrSell',1)
            else:
                df.set_value( i,'BuyOrSell',0)
    df.dropna(inplace=True)
    
    return df

In [6]:
my_df = generate_training_col(my_df)

In [7]:
ezmongo.set_data(my_df, 'MSFT', 'train')

True